# Timer

In [1]:
from time import sleep
from datetime import datetime

month, day, hour, minute = 11, 10, 22, 12

while True:
    dt = datetime.now()
    if dt.month >= month:
        if dt.day > day or dt.month > month:
            break
        if dt.day == day and (dt.hour > hour or (dt.hour == hour and dt.minute > minute)):
            break
    sleep(60)

# Parameter optimization

In [2]:
import sys
import pandas as pd
import numpy as np
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from bot.bot import SigBot
from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)

ttype = 'sell'
pattern = ['PriceChange', 'LinearReg']
work_timeframe = '15m'
higher_timeframe = '4h'
opt_limit = 50
load = False

print(f'Timeframe is {work_timeframe}/{higher_timeframe}, trade type is {ttype}')

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {'PriceChange': {"low_price_quantile": [1 * (i + 1) for i in range(20)]}, 
              'LinearReg': {'timeperiod': [16, 20, 24, 28, 32, 36, 40, 44, 48], 'low_bound': [0]}}

opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load)

stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
if not stat_list:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
else:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')

Timeframe is 15m/4h, trade type is sell
Number of combinations is 180


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 180/180 [11:33<00:00,  3.85s/it]


# Check local statistics

In [3]:
# 15m/4h
stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['forecast_rank'] = (stat['pct_right_forecast_avg'] - 70) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
stat.sort_values('pct_right_forecast_avg', ascending=False).head(20)

,pattern,PriceChange_low_price_quantile,LinearReg_timeperiod,LinearReg_low_bound,pct_right_forecast_1,pct_right_forecast_2,pct_right_forecast_3,pct_right_forecast_4,pct_right_forecast_5,pct_right_forecast_6,pct_right_forecast_7,pct_right_forecast_8,pct_right_forecast_9,pct_right_forecast_10,pct_right_forecast_11,pct_right_forecast_12,pct_right_forecast_13,pct_right_forecast_14,pct_right_forecast_15,pct_right_forecast_16,pct_right_forecast_17,pct_right_forecast_18,pct_right_forecast_19,pct_right_forecast_20,pct_right_forecast_21,pct_right_forecast_22,pct_right_forecast_23,pct_right_forecast_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24,forecasts_num,pct_right_forecast_avg,pct_price_diff_avg,forecast_rank,price_rank
13,PriceChange_LinearReg,2,32,0,100.0,100.00,100.00,100.00,94.12,100.00,88.24,82.35,82.35,82.35,82.35,82.35,82.35,82.35,82.35,82.35,82.35,82.35,88.24,82.35,76.47,64.71,70.59,64.71,-3.69,-4.55,-4.90,-3.76,-4.66,-4.61,-4.86,-5.73,-6.22,-6.73,-6.67,-7.10,-6.54,-6.83,-6.48,-5.31,-5.89,-5.49,-5.29,-5.05,-4.02,-0.62,-0.74,-1.64,17,84.803333,-4.890833,251.656667,-83.144167
15,PriceChange_LinearReg,2,40,0,100.0,100.00,100.00,100.00,94.12,100.00,88.24,82.35,82.35,82.35,82.35,82.35,82.35,82.35,82.35,82.35,82.35,82.35,88.24,82.35,76.47,64.71,70.59,64.71,-3.69,-4.55,-4.90,-3.76,-4.66,-4.61,-4.86,-5.73,-6.22,-6.73,-6.67,-7.10,-6.54,-6.83,-6.48,-5.31,-5.89,-5.49,-5.29,-5.05,-4.02,-0.62,-0.74,-1.64,17,84.803333,-4.890833,251.656667,-83.144167
12,PriceChange_LinearReg,2,28,0,100.0,100.00,100.00,100.00,94.12,100.00,88.24,82.35,82.35,82.35,82.35,82.35,82.35,82.35,82.35,82.35,82.35,82.35,88.24,82.35,76.47,64.71,70.59,64.71,-3.69,-4.55,-4.90,-3.76,-4.66,-4.61,-4.86,-5.73,-6.22,-6.73,-6.67,-7.10,-6.54,-6.83,-6.48,-5.31,-5.89,-5.49,-5.29,-5.05,-4.02,-0.62,-0.74,-1.64,17,84.803333,-4.890833,251.656667,-83.144167
11,PriceChange_LinearReg,2,24,0,100.0,100.00,100.00,100.00,94.12,100.00,88.24,82.35,82.35,82.35,82.35,82.35,82.35,82.35,82.35,82.35,82.35,82.35,88.24,82.35,76.47,64.71,70.59,64.71,-3.69,-4.55,-4.90,-3.76,-4.66,-4.61,-4.86,-5.73,-6.22,-6.73,-6.67,-7.10,-6.54,-6.83,-6.48,-5.31,-5.89,-5.49,-5.29,-5.05,-4.02,-0.62,-0.74,-1.64,17,84.803333,-4.890833,251.656667,-83.144167
9,PriceChange_LinearReg,2,16,0,100.0,100.00,100.00,100.00,94.12,100.00,88.24,82.35,82.35,82.35,82.35,82.35,82.35,82.35,82.35,82.35,82.35,82.35,88.24,82.35,76.47,64.71,70.59,64.71,-3.69,-4.55,-4.90,-3.76,-4.66,-4.61,-4.86,-5.73,-6.22,-6.73,-6.67,-7.10,-6.54,-6.83,-6.48,-5.31,-5.89,-5.49,-5.29,-5.05,-4.02,-0.62,-0.74,-1.64,17,84.803333,-4.890833,251.656667,-83.144167
16,PriceChange_LinearReg,2,44,0,100.0,100.00,100.00,100.00,93.75,100.00,87.50,81.25,81.25,81.25,81.25,81.25,81.25,81.25,81.25,81.25,81.25,81.25,87.50,81.25,75.00,68.75,75.00,68.75,-3.69,-5.00,-5.04,-4.22,-4.90,-4.77,-4.93,-5.77,-6.15,-6.58,-6.63,-7.04,-6.20,-6.66,-6.42,-5.12,-5.88,-5.48,-5.21,-5.14,-4.24,-1.51,-1.93,-2.32,16,84.635417,-5.034583,234.166667,-80.553333
17,PriceChange_LinearReg,2,48,0,100.0,100.00,100.00,100.00,93.33,100.00,86.67,80.00,80.00,80.00,80.00,80.00,80.00,80.00,80.00,80.00,80.00,80.00,86.67,80.00,73.33,66.67,73.33,66.67,-3.69,-4.55,-5.17,-4.68,-5.15,-4.93,-4.86,-5.82,-6.08,-6.44,-6.58,-6.97,-5.85,-6.49,-6.37,-4.93,-5.87,-5.47,-5.13,-5.05,-4.02,-2.40,-3.12,-3.00,15,83.611250,-5.109167,204.168750,-76.637500
10,PriceChange_LinearReg,2,20,0,100.0,94.44,94.44,94.44,94.44,94.44,88.89,83.33,83.33,83.33,83.33,83.33,83.33,83.33,77.78,77.78,83.33,77.78,83.33,77.78,72.22,61.11,66.67,61.11,-3.57,-4.44,-4.38,-3.64,-4.33,-3.97,-4.69,-5.77,-6.15,-6.58,-6.63,-7.04,-6.20,-6.66,-6.42,-5.12,-5.88,-5.48,-5.21,-4.79,-3.63,-0.41,-0.70,

# Check global statistics

In [3]:
from glob import glob

ttype = 'sell'
pattern = ['PriceChange']
work_timeframe = '5m'
higher_timeframe = '1h'
stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
stat = None

for sl in stat_list:
    tmp = pd.read_pickle(sl)
    if stat is None:
        stat = tmp.copy()
    else:
        stat = pd.concat([stat, tmp])
        
stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['forecast_rank'] = (stat['pct_right_forecast_avg'] - 70) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
        
total_stat = stat.groupby(['PriceChange_low_price_quantile']).agg({'pct_right_forecast_avg': 'mean',
                                                                   'pct_price_diff_avg': 'mean',
                                                                   'forecast_rank': 'mean', 
                                                                   'price_rank': 'mean', 
                                                                   'forecasts_num': 'sum'}).sort_values('forecast_rank', 
                                                                                                        ascending=False)
total_stat.head(20)

,pct_right_forecast_avg,pct_price_diff_avg,forecast_rank,price_rank,forecasts_num
PriceChange_low_price_quantile,,,,,
20,72.930243,-1.306944,-879.743681,-157.711944,3071
19,73.131424,-1.325521,-881.025174,-153.987604,3038
17,73.016458,-1.380833,-906.773576,-147.793403,2972
18,72.718576,-1.329062,-908.998681,-150.019097,3005
16,73.201181,-1.421007,-909.151806,-143.919722,2933
15,73.250174,-1.485451,-924.197604,-142.200174,2911
12,74.196285,-1.676250,-927.744236,-130.194201,2805
13,73.203299,-1.593021,-954.058576,-131.837604,2836
11,73.823993,-1.782292,-954.826875,-126.921111,2770


# Save new config data to config file

In [7]:
from config_updater import ConfigUpdater

ttype = 'sell'
pattern = ['PriceChange']
work_timeframe = '15m'
higher_timeframe = '4h'
timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {'PriceChange': {"low_price_quantile": [7]}, 
              'LinearReg': {'timeperiod': [24], 'low_bound': [0]}}
        
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)